## P2P Lending Dataset Preparation

This script has the goal of merging the data made available from [Lending Club](https://www.lendingclub.com) between 2007 and 2016.
In this work, we tackle only "Charged Off" and "Fully Paid" loans.
The main steps taken to prepare the dataset are the following:

1. Data load and header sanity check
2. Data concatenation
3. Removal and treatment of string variables
4. Removal of instances (loan requests) with many missing values
5. Removal of features (attributes) with many missing values
6. Removal of features to avoid data leakage

### 1. Data load and header sanity check.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df2007to2011 = pd.read_csv("./LoanStats3a_securev1.csv", low_memory=False, skiprows=[0])
df2012to2013 = pd.read_csv("./LoanStats3b_securev1.csv", low_memory=False, skiprows=[0])
df2014       = pd.read_csv("./LoanStats3c_securev1.csv", low_memory=False, skiprows=[0])
df2015       = pd.read_csv("./LoanStats3d_securev1.csv", low_memory=False, skiprows=[0])
df2016Q1     = pd.read_csv("./LoanStats_securev1_2016Q1.csv", low_memory=False, skiprows=[0])
df2016Q2     = pd.read_csv("./LoanStats_securev1_2016Q2.csv", low_memory=False, skiprows=[0])
df2016Q3     = pd.read_csv("./LoanStats_securev1_2016Q3.csv", low_memory=False, skiprows=[0])
df2016Q4     = pd.read_csv("./LoanStats_securev1_2016Q4.csv", low_memory=False, skiprows=[0])

all_dfs = [df2007to2011, df2012to2013, df2014, df2015, df2016Q1, df2016Q2, df2016Q3, df2016Q4]

#### Checking out how the data is shaped and if they match

In [3]:
columnsFirstDF = list(all_dfs[0].columns.values)
for df in all_dfs:
    if set(df.columns.values) != set(columnsFirstDF):
        print("Things are not maching!")
#     print(df.shape)

### 2. Data concatenation

In [4]:
df = pd.concat(all_dfs)
print(df.shape)

(1321864, 128)


#### Filters dataset to contain only "Charged Off" and "Fully Paid" loans

In [8]:
df = df.loc[(df.loan_status == "Charged Off") | (df.loan_status == "Fully Paid")]
print(df.shape)

(578331, 128)


### 3. Removal and treatment of string variables

In [10]:
df_string = df.select_dtypes(exclude=[np.number])
print(df_string.shape)
pd.set_option('display.max_columns', 30)
display(df_string.head())

(578331, 26)


,id,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint
0,1077501,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,Jan-1985,83.7%,f,Jan-2015,NaN,Apr-2017,INDIVIDUAL,NaN
1,1077430,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,Apr-1999,9.4%,f,Apr-2013,NaN,Oct-2016,INDIVIDUAL,NaN
2,1077175,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,Nov-2001,98.5%,f,Jun-2014,NaN,Apr-2017,INDIVIDUAL,NaN
3,1076863,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,Feb-1996,21%,f,Jan-2015,NaN,Apr-2016,INDIVIDUAL,NaN
4,1075358,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,Jan-1996,53.9%,f,Jan-2017,NaN,Jan-2017,INDIVIDUAL,NaN


### 4. Removal of instances (loan requests) with many missing values

In [11]:
#### Getting rid of instances with too many missing values (above 90%)
df.dropna(thresh = 0.1 * df.shape[1], axis = 0, inplace = True)
display(df.shape)

(578331, 128)

### 5. Removal of features (attributes) with many missing values

In [12]:
#### Removes all features with more than 70% of the values missing
df.dropna(thresh = 0.3 * df.shape[0], axis = 1, inplace = True)
display(df.shape)

(578331, 92)

### 6. Removal of features to avoid data leakage

In [13]:
featuresToRemove = ['id',
                    'issue_d',
                    'last_credit_pull_d',
                    'last_pymnt_amnt',
                    'last_pymnt_d',
                    'member_id',
                    'next_pymnt_d',
                    'policy_code',
                    'revol_bal',
                    'revol_bal_joint',
                    'revol_util',
                    'title',
                    'total_pymnt',
                    'total_pymnt_inv',
                    'total_rec_int',
                    'total_rec_late_fee',
                    'total_rec_prncp',
                    'url']
df.drop(featuresToRemove, axis = 1, inplace = True)
display(df.head())

ValueError: labels ['member_id' 'next_pymnt_d' 'revol_bal_joint'] not contained in axis

## Saves this final DF to a csv file

In [14]:
df.to_csv("./p2p_lendingclub_clean.csv")

# OTHER STUFF

#### Converting n/a into NaNs and NaNs into 0s

In [9]:
df.replace('n/a', np.nan, inplace = True)
df.replace(np.nan, 0.0, inplace = True)

#### Dropping irrelevant of features that would strongly bias our models

#### Checking data types

In [11]:
# display(df.dtypes)

#### Checking how many missing values we have per feature

In [12]:
# display(df.isnull().sum() / df.shape[0])

loan_amnt                              0.0
funded_amnt                            0.0
funded_amnt_inv                        0.0
term                                   0.0
int_rate                               0.0
installment                            0.0
grade                                  0.0
sub_grade                              0.0
emp_title                              0.0
emp_length                             0.0
home_ownership                         0.0
annual_inc                             0.0
verification_status                    0.0
loan_status                            0.0
pymnt_plan                             0.0
desc                                   0.0
purpose                                0.0
zip_code                               0.0
addr_state                             0.0
dti                                    0.0
delinq_2yrs                            0.0
earliest_cr_line                       0.0
fico_range_low                         0.0
fico_range_

#### Getting rid of non-numeric data

In [15]:
# df_num = df.select_dtypes(include=[np.number])
# display(df_num.shape)
# display(df_num)

(1321864, 92)

#### Checking the correlation between features

In [16]:
df_correlation = df.corr().abs()
#display(df_correlation)


import seaborn as sns
sns.heatmap(df_correlation,
            xticklabels=df_correlation.columns.values,
            yticklabels=df_correlation.columns.values)

#### Let's take a look at string data

(1321864, 18)


,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,loan_status,pymnt_plan,desc,purpose,zip_code,addr_state,earliest_cr_line,initial_list_status,application_type,verification_status_joint
0,36 months,10.65%,B,B2,0,10+ years,RENT,Verified,Fully Paid,n,Borrower added on 12/22/11 > I need to upgra...,credit_card,860xx,AZ,Jan-1985,f,INDIVIDUAL,0
1,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Charged Off,n,Borrower added on 12/22/11 > I plan to use t...,car,309xx,GA,Apr-1999,f,INDIVIDUAL,0
2,36 months,15.96%,C,C5,0,10+ years,RENT,Not Verified,Fully Paid,n,0,small_business,606xx,IL,Nov-2001,f,INDIVIDUAL,0
3,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Fully Paid,n,Borrower added on 12/21/11 > to pay for prop...,other,917xx,CA,Feb-1996,f,INDIVIDUAL,0
4,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Fully Paid,n,Borrower added on 12/21/11 > I plan on combi...,other,972xx,OR,Jan-1996,f,INDIVIDUAL,0
5,36 months,7.90%,A,A4,Veolia Transportaton,3 years,RENT,Source Verified,Fully Paid,n,0,wedding,852xx,AZ,Nov-2004,f,INDIVIDUAL,0
6,60 months,15.96%,C,C5,Southern Star Photography,8 years,RENT,Not Verified,Fully Paid,n,Borrower added on 12/18/11 > I am planning o...,debt_consolidation,280xx,NC,Jul-2005,f,INDIVIDUAL,0
7,36 months,18.64%,E,E1,MKC Accounting,9 years,RENT,Source Verified,Fully Paid,n,Borrower added on 12/16/11 > Downpayment for...,car,900xx,CA,Jan-2007,f,INDIVIDUAL,0
8,60 months,21.28%,F,F2,0,4 years,OWN,Source Verified,Charged Off,n,Borrower added on 12/21/11 > I own a small h...,small_business,958xx,CA,Apr-2004,f,INDIVIDUAL,0
9,60 months,12.69%,B,B5,Starbucks,< 1 year,RENT,Verified,Charged Off,n,Borrower added on 12/16/11 > I'm trying to b...,other,774xx,TX,Sep-2004,f,INDIVIDUAL,0


#### Runs a decision tree to see what's going on

In [55]:
# from sklearn import tree
# from sklearn.model_selection import train_test_split

# X = df_num
# y = df['loan_status']

# xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.33, random_state=42)

# clf = tree.DecisionTreeClassifier()
# clf.fit(xTrain, yTrain)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [60]:
# df.groupby('loan_status').count()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,desc,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
loan_status,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
Charged Off,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,...,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324,117324
Current,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,...,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753,707753
Default,159,159,159,159,159,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159,159,159,159,159,159
Does not meet the credit policy. Status:Charged Off,761,761,761,761,761,761,761,761,761,761,761,761,761,761,761,...,761,761,761,761,761,761,761,761,761,761,761,761,761,761,761
Does not meet the credit policy. Status:Fully Paid,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,...,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988,1988
Fully Paid,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,...,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007,461007
In Grace Period,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,...,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267,10267
Late (16-30 days),4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,...,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002,4002


In [57]:
# from IPython.display import Image  
# import pydotplus 
# dot_data = tree.export_graphviz(clf, out_file=None, 
#                          feature_names=X.columns.values,  
#                          class_names=y,  
#                          filled=True, rounded=True,  
#                          special_characters=True)  
# graph = pydotplus.graph_from_dot_data(dot_data)  
# Image(graph.create_png())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().